In [1]:
# Imports
import pandas as pd
import plotly.express as px 
import ipaddress
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# read connection traffic data
df = pd.read_csv (r'/Users/ZOHELHAS/Desktop/connection-traffic-analysis/bin0.csv', encoding = 'unicode_escape')
df = df.dropna()
display(df)

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,192.168.137.35,239.255.255.250,SSDP,217,M-SEARCH * HTTP/1.1
1,2,1.257717,192.168.137.190,185.60.216.15,TLSv1.2,98,Application Data
2,3,1.285469,185.60.216.15,192.168.137.190,TLSv1.2,94,Application Data
3,4,1.288602,192.168.137.190,185.60.216.15,TCP,66,41894 > 443 [ACK] Seq=33 Ack=29 Win=501 Len=...
4,5,1.467483,147.161.177.193,192.168.137.35,TCP,60,443 > 49978 [ACK] Seq=1 Ack=1 Win=2061 Len=0
...,...,...,...,...,...,...,...
753571,753572,4968.934199,192.168.137.1,224.0.0.252,LLMNR,71,Standard query 0x6a9a A raspberrypi
753572,753573,4969.261519,192.168.137.1,192.168.137.255,NBNS,92,Name query NB RASPBERRYPI<00>
753573,753574,4969.514412,192.168.137.1,224.0.0.251,MDNS,77,"Standard query 0x0000 A raspberrypi.local, ""QM..."
753574,753575,4969.515214,fe80::e824:3b27:66ba:d604,ff02::fb,MDNS,97,"Standard query 0x0000 A raspberrypi.local, ""QM..."


In [3]:
 # check if string is a valid ip address or not
 def is_ip(string):
        try:
            ipaddress.ip_address(string)
            return True
        except ValueError:
            return False

In [11]:
def filter_fn(row):
    if (is_ip(row['Destination'])) and (ipaddress.ip_address(row['Destination']).is_private):
        return True
    else:
        return False

m = df.apply(filter_fn, axis=1)

private_request_df = df[m]
public_request_df = df[~m]

In [ ]:
import ipaddress
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

lookup_table = [
    ('Google',' https://raw.githubusercontent.com/lord-alfred/ipranges/main/google/ipv4.txt'),
    ('Google',' https://raw.githubusercontent.com/lord-alfred/ipranges/main/google/ipv6.txt'),
    ('Bing','https://raw.githubusercontent.com/lord-alfred/ipranges/main/bing/ipv4.txt'),
    ('Amazon','https://raw.githubusercontent.com/lord-alfred/ipranges/main/amazon/ipv4.txt'),
    ('Amazon','https://raw.githubusercontent.com/lord-alfred/ipranges/main/amazon/ipv6.txt'),
    ('Microsoft','https://raw.githubusercontent.com/lord-alfred/ipranges/main/microsoft/ipv4.txt'),
    ('Microsoft','https://raw.githubusercontent.com/lord-alfred/ipranges/main/microsoft/ipv6.txt'),
    ('Oracle','https://raw.githubusercontent.com/lord-alfred/ipranges/main/oracle/ipv4.txt'),
    ('DigitalOcean','https://raw.githubusercontent.com/lord-alfred/ipranges/main/digitalocean/ipv4.txt'),
    ('DigitalOcean','https://raw.githubusercontent.com/lord-alfred/ipranges/main/digitalocean/ipv6.txt')
]

for index, tuple in enumerate(lookup_table):

    target_url = tuple[1]
    cloud_name = tuple[0]

    for line in urlopen(target_url):
        value = line.decode('utf-8').strip()
        ipList = public_request_df['Destination'].values
        for elem in ipList:
            if is_ip(elem):
                if ipaddress.ip_address(elem) in ipaddress.ip_network(value):
                    public_request_df['Cloud_Service'] = cloud_name


Match public DNS Service IP Address to rows in the dataframe

In [ ]:
df.loc[df['Source'].str.contains("8.8.8.8"),'DNS_Service'] = 'Google'
df.loc[df['Source'].str.contains("8.8.4.4"),'DNS_Service'] = 'Google'
df.loc[df['Source'].str.contains("9.9.9.9"),'DNS_Service'] = 'Quad9'
df.loc[df['Source'].str.contains("149.112.112.112"),'DNS_Service'] = 'Quad9'
df.loc[df['Source'].str.contains("208.67.222.222"),'DNS_Service'] = 'OpenDNS'
df.loc[df['Source'].str.contains("208.67.220.220"),'DNS_Service'] = 'OpenDNS'
df.loc[df['Source'].str.contains("1.1.1.1"),'DNS_Service'] = 'Cloudflare'
df.loc[df['Source'].str.contains("1.0.0.1"),'DNS_Service'] = 'Cloudflare'
df.loc[df['Source'].str.contains("185.228.168.9"),'DNS_Service'] = 'CleanBrowsing'
df.loc[df['Source'].str.contains("185.228.169.9"),'DNS_Service'] = 'CleanBrowsing'
df.loc[df['Source'].str.contains("76.76.19.19"),'DNS_Service'] = 'Alternate DNS'
df.loc[df['Source'].str.contains("76.223.122.150"),'DNS_Service'] = 'Alternate DNS'
df.loc[df['Source'].str.contains("94.140.14.14"),'DNS_Service'] = 'AdGuard DNS'
df.loc[df['Source'].str.contains("94.140.15.15"),'DNS_Service'] = 'AdGuard DNS'

#google = df.loc[df['Source'].str.contains("8.8.8.8", case=False)]
display(df)

IP Version Analysis

In [ ]:
df.loc[df['Source'].str.contains("."),'IP_Version'] = 'ipv4'
df.loc[df['Source'].str.contains(":"),'IP_Version'] = 'ipv6'


In [ ]:
fig = px.histogram(df, x=df['IP_Version'], marginal = "box",width=800, height=400)
fig.show()

In [ ]:
#fig = px.scatter(df, x='Time', y='Source', color='Protocol')
#fig.show()

Protocol Analysis

In [ ]:
fig = px.box(df, x='Length', color='Protocol')
fig.show()

In [ ]:
fig = px.histogram(df, x=df['Length'], marginal = "box",width=800, height=400)
fig.show()

In [ ]:
fig = px.histogram(df, x=df['Protocol'], marginal = "box",width=800, height=400)
fig.show()